In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# 1. 데이터 로드
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

def transform_data(df):
    # A. Title 추출
    df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
    df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    df['Title'] = df['Title'].replace(['Mlle', 'Ms'], 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    df['Title'] = df['Title'].map(title_mapping).fillna(0)
    
    # B. 결측치 처리 (중앙값/최빈값)
    df['Age'] = df['Age'].fillna(train_df['Age'].median())
    df['Fare'] = df['Fare'].fillna(train_df['Fare'].median())
    df['Embarked'] = df['Embarked'].fillna('S')
    
    # C. 수치화 및 인코딩
    df['Sex'] = df['Sex'].map({'female': 1, 'male': 0})
    df['Embarked'] = df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
    
    # D. 새로운 피처 생성 (FamilySize, IsAlone)
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    
    # E. [전략 1] Deck 추출 (Cabin의 첫 글자)
    df['Deck'] = df['Cabin'].str.slice(0, 1).fillna('U')
    deck_mapping = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "T": 8, "U": 0}
    df['Deck'] = df['Deck'].map(deck_mapping)
    
    # F. [전략 2] Age * Pclass 상호작용 피처
    df['Age_Class'] = df['Age'] * df['Pclass']
    
    return df

# 전처리 실행
train_data = transform_data(train_df)
test_data = transform_data(test_df)

# 학습 데이터 구성
features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'FamilySize', 'IsAlone', 'Title', 'Deck', 'Age_Class']
X = train_data[features]
y = train_data['Survived']

# 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# [Step 3] 아까 찾은 최적의 하이퍼파라미터 적용 (GridSearch 결과 반영)
# n_estimators, max_depth 등은 사용자님이 아까 얻은 best_params_ 값을 넣으시면 됩니다.
final_rf = RandomForestClassifier(
    n_estimators=200, 
    max_depth=7, 
    min_samples_leaf=1, 
    min_samples_split=5, 
    random_state=42
)

final_rf.fit(X_train, y_train)

# 검증 점수 확인
final_score = final_rf.score(X_val, y_val) * 100
print(f"🚀 최종 검증 정확도: {final_score:.2f}%")

# [Step 4] 제출 파일 생성
predictions = final_rf.predict(test_data[features])
submission = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": predictions
})
submission.to_csv('submission_final_85.csv', index=False)
print("✅ 제출 파일(submission_final_85.csv)이 생성되었습니다!")